In [1]:
import os
import numpy as np
import pandas
import requests, sys, zipfile, io
import pickle

In [9]:
data_dir = 'bank(3)/'

# n= 6000
# K = 4
K = 10
_path = 'bank-additional-full.csv' # Big dataset with 41108 samples
# _path = 'bank.csv' # most approaches use this small version with 4521 samples
data_path = os.path.join(data_dir, _path)
if (not os.path.exists(data_path)): 

    print('Bank dataset does not exist in current folder --- Have to download it')
    r = requests.get('https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip', allow_redirects=True)
    # r = requests.get('https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip', allow_redirects=True)
    if r.status_code == requests.codes.ok:
        print('Download successful')
    else:
        print('Could not download - please download')
        sys.exit()

    z = zipfile.ZipFile(io.BytesIO(r.content))
    # z.extract('bank-additional/bank-additional-full.csv','./data')
    open(data_path, 'wb').write(z.read('bank-additional/bank-additional-full.csv'))
    # open(data_path, 'wb').write(z.read('bank.csv'))

df = pandas.read_csv(data_path,sep=';')
print(df.columns)
#        shape = df.shape

#        df = df.loc[np.random.choice(df.index, n, replace=False)]
sex = df['marital'].astype(str).values
sens_attributes = list(set(sex))

if 'unknown' in sens_attributes:
    sens_attributes.remove('unknown')

df1 = df.loc[df['marital'] == sens_attributes[0]]
df2 = df.loc[df['marital'] == sens_attributes[1]]
df3 = df.loc[df['marital'] == sens_attributes[2]]

df = [df1, df2, df3]
df = pandas.concat(df)
sex = df['marital'].astype(str).values
labels = (df['y'].astype(str).values == 'yes').astype(np.int64)

df = df[['age','duration','euribor3m', 'nr.employed', 'cons.price.idx', 'campaign']].values
# df = df[['age','duration','balance']].values

sens_attributes = list(set(sex))
sex_num = np.zeros(df.shape[0], dtype=int)
sex_num[sex == sens_attributes[1]] = 1
sex_num[sex == sens_attributes[2]] = 2

binary_sex_num = np.zeros(df.shape[0], dtype=int)
binary_sex_num[sex == sens_attributes[1]] = 1
binary_sex_num[sex == sens_attributes[2]] = 1

inputs = np.array(df, dtype=float)

n = inputs.shape[0]
sensitives = sex_num
binary_sensitives = binary_sex_num

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')


In [ ]:
""" full or subsampling (three groups) """
subsampling = False
subsample_size = 1000
if not subsampling:
    np.save('bank(3)/inputs.npy', inputs)
    np.save('bank(3)/labels.npy', labels)
    np.save('bank(3)/sensitives.npy', sensitives)
else:
    subsample_ids = np.random.choice(n, size=subsample_size, replace=False) # 1000 samples
    # subsample_ids0 = np.random.choice(np.where(sensitives ==0)[0], size=500, replace=False)
    # subsample_ids1 = np.random.choice(np.where(sensitives ==1)[0], size=500, replace=False)
    # subsample_ids = np.concatenate([subsample_ids0, subsample_ids1])
    inputs_subsampled, sensitives_subsampled, labels_subsampled = inputs[subsample_ids], sensitives[subsample_ids], labels[subsample_ids]
    np.save(f'bank(3)/inputs_subsampled_{subsample_size}.npy', inputs)
    np.save(f'bank(3)/labels_subsampled_{subsample_size}.npy', labels)
    np.save(f'bank(3)/sensitives_subsampled_{subsample_size}.npy', sensitives)
print(inputs.shape, labels.shape, sensitives.shape)

(41108, 6) (41108,) (41108,)


In [10]:
""" full or subsampling (two groups) """
subsampling = False
subsample_size = 1000
if not subsampling:
    np.save('bank(2)/inputs.npy', inputs)
    np.save('bank(2)/labels.npy', labels)
    np.save('bank(2)/sensitives.npy', binary_sensitives)
else:
    subsample_ids = np.random.choice(n, size=subsample_size, replace=False) # 1000 samples
    # subsample_ids0 = np.random.choice(np.where(sensitives ==0)[0], size=500, replace=False)
    # subsample_ids1 = np.random.choice(np.where(sensitives ==1)[0], size=500, replace=False)
    # subsample_ids = np.concatenate([subsample_ids0, subsample_ids1])
    inputs_subsampled, sensitives_subsampled, labels_subsampled = inputs[subsample_ids], binary_sensitives[subsample_ids], labels[subsample_ids]
    np.save(f'bank(2)/inputs_subsampled_{subsample_size}.npy', inputs)
    np.save(f'bank(2)/labels_subsampled_{subsample_size}.npy', labels)
    np.save(f'bank(2)/sensitives_subsampled_{subsample_size}.npy', binary_sensitives)
print(inputs.shape, labels.shape, binary_sensitives.shape)

(41108, 6) (41108,) (41108,)
